# 活性予測モデルの解釈
**(注)　本セクションはハンズオンの時間を余らせた方のためのコンテンツとして用意したものです。**

ここでは[SHAP](https://shap.readthedocs.io/en/latest/)を用いて活性予測モデルの解釈を行います。
予測モデルは2-1で利用したものを使います。
SHAPは

pip install shapでインストールしておいてください

## 予測モデル構築のために利用するライブラリのインポート
- 今回はLightGBMの回帰モデルを利用し、サポートベクター回帰モデルとの比較も行います。
- ハイパーパラメータの最適化にはOptunaを利用します。

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from useful_rdkit_utils import mol2numpy_fp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import optuna
# Logging levelを変えておきます
optuna.logging.set_verbosity(optuna.logging.ERROR)

from lightgbm import LGBMRegressor
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import joblib

import pathlib
import sys
import os
#　実行するノートブックのパスを取得します
notedir = pathlib.Path().resolve()
print(notedir)

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.info')

### 後の描画用にユーティリティ関数を定義しておきます

In [ ]:
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import rdDepictor

def mol2svg(mol):
    rdDepictor.Compute2DCoords(mol)
    d2d = rdMolDraw2D.MolDraw2DSVG(200, 100)
    d2d.DrawMolecule(mol)
    d2d.FinishDrawing()
    return d2d.GetDrawingText()

## DPP４データの読み込み

予測モデルの構築に使うデータを読み込みます。

In [ ]:
df = pd.read_table('./dpp4_valid.tsv', sep='\t')
df.head(5)

In [ ]:
# データの大きさの確認
print(df.shape)

### 塩の取り扱い
データセットに塩を含む分子が含まれているのでモデル構築前に正規化が必要です。01のチュートリアルのコードを利用します。

**ここで復習をしましょう**

In [ ]:
# 塩を含むデータの確認
for smi in df['Smiles']:
    if '.' in smi:
        print(smi)

parent_dir = os.path.abspath(os.path.join(notedir, os.pardir))
# cheminfo_util をimportします。
sys.path.append(parent_dir)
import cheminfo_util

In [ ]:
# 分子の正規化と合わせてKiの値をpKiに変換します
df['ROMol'] = df['Smiles'].apply(Chem.MolFromSmiles)
df['clean_mol'] = df['ROMol'].apply(cheminfo_util.prep_moleclue) # ここで分子の正規化を実行しています
df['pKi'] = df['Standard Value'].apply(lambda x: 9-np.log10(x))

確認のために最初の５０化合物を表示させてみます。

In [ ]:
Draw.MolsToGridImage(df['clean_mol'][:50], molsPerRow=5)

##　フィンガープリントの生成

In [ ]:
# 描画用
clean_mols = df['clean_mol'].to_list()
mols_svgs = [mol2svg(m) for m in clean_mols]
X = np.array([mol2numpy_fp(m, 2, 1024) for m in df['clean_mol']])
y = np.array([float(v) for v in df['pKi']]).ravel()
print(X.shape, y.shape)

### 訓練セット、テストセットの分割
訓練用のデータとテスト用のデータに分割するためにランダムスプリットをおこないます。全データのうち７０%を訓練データに利用し、残りの30%を性能確認のためのテストデータとします。

In [ ]:
train_idx, test_idx = train_test_split([i for i in range(X.shape[0])], train_size=0.7, random_state=111)

In [ ]:
train_X = X[train_idx]
train_svg = [mols_svgs[i] for i in train_idx]
test_X = X[test_idx]
test_svg = [mols_svgs[i] for i in test_idx]
train_y = y[train_idx]
test_y = y[test_idx]

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

### ハイパーパラメータチューニングを行う
Objective functionの定義
- optunaでハイパーパラメータの最適化を行うためにはobjective関数の定義が必要です。
- 以下のコードでは[cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)で得られるr2の平均値を評価用の値に利用しています。
- チューニングするハイパーパラメータは、そのサンプリングの仕方によって範囲と、[サンプリングメソッド](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial)を変更します。
  - suggest_int 整数をサンプリング
  - suggest_loguniform, 対数一様分布からのサンプリング suggest_float(log=True)が推奨される
  - suggest_uniform　一様分布からのサンプリング suggest_float()が推奨される
  - suggest_categorical　カテゴリ変数からのサンプリング
  - etc.

In [ ]:
def objective(trial, x, t, cv):
    # 1. 最適化するパラメータを設定します
    # https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
    n_estimaters = trial.suggest_int('n_estimators', 1, 100) #Boost Treeの数
    max_depth = trial.suggest_int('max_depth', 1, 10) #探索木の深さ
    num_leaves = trial.suggest_int('num_leaves', 2, 10)
    min_child_weight = trial.suggest_float("min_child_weight", 0.1, 10, log=True)
    subsample = trial.suggest_float("subsample",0.55, 0.95)    
    colsample_bytree = trial.suggest_float("subsample",0.55, 0.95)    

    # 2. LightGBMREgressor
    estimator = LGBMRegressor(
        n_estimators=n_estimaters,
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree= colsample_bytree,
        random_state=111,
        verbose=-1
    )

    # 3. 学習の実行、検証結果の表示
    print('Current_params : ', trial.params)
    r2 = cross_val_score(estimator, x, t, cv=cv, scoring="r2").mean()
    print(r2)
    print("#######")
    return r2

In [ ]:
# r2の最適化なので方向性は最大となるように設定します
study = optuna.create_study(direction='maximize')
cv = 10

In [ ]:
# n_trials 50だと時間が少しかかるかもしれません。実行したら、しばし休憩しましょう。
study.optimize(lambda trial: objective(trial, train_X, train_y, cv), n_trials=50)

### 最適化後のR2値は0．6前後でした。実際にテストデータをプロットして確認してみましょう

In [ ]:
print(study.best_value)

Optunaの結果得られた最良のハイパーパラメータを用いて予測モデルを構築します。

In [ ]:
best_lgbm = LGBMRegressor(**study.best_params)
best_lgbm.fit(train_X, train_y)

In [ ]:
pred_y = best_lgbm.predict(test_X)
pred_train_y = best_lgbm.predict(train_X)

## モデルの性能の視覚化
予測モデルの性能を確認するため、「訓練データの予測結果」「テストセットの予測結果」を実測値に比べてどのくらいズレているかをプロットします。

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.title('LightGBM model for DPP4 activity prediction')
plt.style.use('ggplot')
plt.scatter(pred_train_y, train_y, alpha=0.8, c='pink')
plt.scatter(pred_y, test_y, alpha=0.4, c='blue')
plt.plot(np.linspace(4,9.5), np.linspace(4,9.5))
plt.xlabel('predicted pKi')
plt.ylabel('acctual pKi')
plt.show()

# モデルの解釈
ここからSHAPを利用してモデルを解釈します。

In [ ]:
import shap
from rdkit.Chem import AllChem

explainer = shap.TreeExplainer(model=best_lgbm, 
                                   feature_perturbation='interventional', 
                                   model_output='raw')
shap_values = explainer(X)

どの特徴（今回はFingerprintなのでX番目の部分構造フラグ）が予測に寄与しているのかをバープロットでみてみます

In [ ]:
shap.summary_plot(shap_values, X, plot_type="bar")

同様にサマリープロットでも確認します。かなり明確に別れますね。

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
m = df['clean_mol'][2] #適当に選びました。数字を変更すれば構造も変わります。
m

Fingerprint生成時にmol2numpy_fpというユーティリティ関数を使いましたが、ビットに対応する部分構造を表示したいので、もう一度計算しなおします。
実践的には予めモデルの解釈時に利用することを見越してinfoを計算しておくことが多いと思います。

In [ ]:
info = {}
#mol2numpy_fp(m, 2, 1024)だったのでradius=2, ビット=1024を指定します。
fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024, bitInfo=info)
#print(info)

infoには「33番目のビットはインデックス０の原子の半径0の部分構造,インデックス13の原子の半径0の部分構造,インデックス14の原子の半径0の部分構造」というような情報がはいっています。これを視覚化すると

In [ ]:
morgan_turples = ((m, k, info) for k in list(info.keys()))
Draw.DrawMorganBits(morgan_turples, molsPerRow=6, legends=['bit: '+str(x) for x in list(info.keys())])

378番目の特徴に関しては[SBDD的な解釈の点](https://numon.pdbj.org/mom/202?l=ja)からも納得感があります。